In [1]:
import pandas as pd 
import os 
PATH_TRAIN = os.path.join("..","data","train_data_ext","train.csv")
PATH_TEST = os.path.join("..","data","train_data_ext","test.csv")
PATH_VAL = os.path.join("..","data","train_data_ext","val.csv")

In [2]:
train_data = pd.read_csv(PATH_TRAIN)
test_data = pd.read_csv(PATH_TEST)
val_data = pd.read_csv(PATH_VAL)

In [3]:
train_data.head()

,text,label
0,"I didn't feel humiliated, which was a surprise...",sadness
1,"I didn't feel humiliated, which was a surprise...",sadness
2,I can go from feeling so hopeless to so damned...,sadness
3,I'm grabbing a minute to post because I feel i...,anger
4,I am ever feeling nostalgic about the fireplac...,love


In [4]:
train_data = train_data.drop_duplicates()

In [5]:
train_data.head()

,text,label
0,"I didn't feel humiliated, which was a surprise...",sadness
2,I can go from feeling so hopeless to so damned...,sadness
3,I'm grabbing a minute to post because I feel i...,anger
4,I am ever feeling nostalgic about the fireplac...,love
5,"I am feeling extremely grouchy today, and to b...",anger


In [6]:
idx_2_class = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
class_2_idx = {value: key for key, value in idx_2_class.items()}


In [7]:
train_data["labels"] = train_data["label"].replace(class_2_idx)
test_data["labels"] = test_data["label"].replace(class_2_idx)
val_data["labels"] = val_data["label"].replace(class_2_idx)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11768\1271200801.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data["labels"] = train_data["label"].replace(class_2_idx)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11768\1271200801.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data["labels"] = test_data["label"].replace(class_2_idx)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11768\1271200801.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the ol

In [8]:
train_data

,text,label,labels
0,"I didn't feel humiliated, which was a surprise...",sadness,0
2,I can go from feeling so hopeless to so damned...,sadness,0
3,I'm grabbing a minute to post because I feel i...,anger,3
4,I am ever feeling nostalgic about the fireplac...,love,2
5,"I am feeling extremely grouchy today, and to b...",anger,3
...,...,...,...
395,I feel incredibly thankful for the lessons I'm...,joy,1
396,I feel such a profound and unshakeable longing...,love,2
397,I feel distinctly called in Clermont to focus ...,love,2
398,I hope you can feel glad that she gave you so ...,joy,1


In [9]:
from datasets import Dataset
ds_train = Dataset.from_pandas(train_data)
ds_test = Dataset.from_pandas(test_data)
ds_val = Dataset.from_pandas(val_data)

In [10]:
ds_train = ds_train.remove_columns(['__index_level_0__'])

In [11]:
ds_val

Dataset({
    features: ['text', 'label', 'labels'],
    num_rows: 200
})

In [12]:
from transformers import (
     AutoTokenizer,
    AutoModelForSequenceClassification ,
)
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id,cache_dir="../modernbert_tokenizer_cache")
device

'cuda'

In [13]:
max_len = 0 
def find_max_len(paragraph):
    global max_len
    len_sentence = len(paragraph)
    max_len = max(max_len, len_sentence)

In [14]:
max_input_length =8024 # 27814
# max_output_length = 8
batch_size = 2

In [15]:
max_len

0

In [16]:
def process_data_to_model_inputs(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

In [17]:
train_dataset = ds_train.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)
val_dataset = ds_val.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
test_dataset = ds_test.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)


In [20]:
train_data

,text,label,labels
0,"I didn't feel humiliated, which was a surprise...",sadness,0
2,I can go from feeling so hopeless to so damned...,sadness,0
3,I'm grabbing a minute to post because I feel i...,anger,3
4,I am ever feeling nostalgic about the fireplac...,love,2
5,"I am feeling extremely grouchy today, and to b...",anger,3
...,...,...,...
395,I feel incredibly thankful for the lessons I'm...,joy,1
396,I feel such a profound and unshakeable longing...,love,2
397,I feel distinctly called in Clermont to focus ...,love,2
398,I hope you can feel glad that she gave you so ...,joy,1


In [21]:
from datasets import Value 
train_dataset = train_dataset.cast_column("labels", Value("int64"))
val_dataset = val_dataset.cast_column("labels", Value("int64"))
test_dataset = test_dataset.cast_column("labels", Value("int64"))

Casting the dataset:   0%|          | 0/399 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [22]:
num_labels = len(idx_2_class)
num_labels

6

In [23]:
from transformers import BitsAndBytesConfig
from peft import prepare_model_for_kbit_training,LoraConfig,PeftModel,get_peft_model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16")

In [24]:
# !pip install --user https://huggingface.co/madbuda/triton-windows-builds/resolve/main/triton-3.0.0-cp310-cp310-win_amd64.whl

In [25]:
import torch
print(torch.__version__)


2.6.0+cu118


In [ ]:
robert_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    cache_dir="../Modern_Bert/modern_bert",
    # quantization_config=bnb_config,          
    id2label=idx_2_class,
    label2id=class_2_idx,
    num_labels=num_labels  
).to(device)

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

f:\python_file_install\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\Panipat\WEPI\aryan\aryan\models\text\Modern_Bert\models--answerdotai--ModernBERT-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at an

In [32]:
for name, module in robert_model.named_modules():
    print(name)



model
model.embeddings
model.embeddings.tok_embeddings
model.embeddings.norm
model.embeddings.drop
model.layers
model.layers.0
model.layers.0.attn_norm
model.layers.0.attn
model.layers.0.attn.Wqkv
model.layers.0.attn.rotary_emb
model.layers.0.attn.Wo
model.layers.0.attn.out_drop
model.layers.0.mlp_norm
model.layers.0.mlp
model.layers.0.mlp.Wi
model.layers.0.mlp.act
model.layers.0.mlp.drop
model.layers.0.mlp.Wo
model.layers.1
model.layers.1.attn_norm
model.layers.1.attn
model.layers.1.attn.Wqkv
model.layers.1.attn.rotary_emb
model.layers.1.attn.Wo
model.layers.1.attn.out_drop
model.layers.1.mlp_norm
model.layers.1.mlp
model.layers.1.mlp.Wi
model.layers.1.mlp.act
model.layers.1.mlp.drop
model.layers.1.mlp.Wo
model.layers.2
model.layers.2.attn_norm
model.layers.2.attn
model.layers.2.attn.Wqkv
model.layers.2.attn.rotary_emb
model.layers.2.attn.Wo
model.layers.2.attn.out_drop
model.layers.2.mlp_norm
model.layers.2.mlp
model.layers.2.mlp.Wi
model.layers.2.mlp.act
model.layers.2.mlp.drop
mod

In [39]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [33]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "Wqkv",    # The combined QKV projection linear layer in attention
        "Wo",      # Attention output projection linear layer
        "mlp.Wi",  # MLP (feed-forward) input linear layer
        "mlp.Wo",  # MLP output linear layer
    ],

)
roberta_model_1 = prepare_model_for_kbit_training(robert_model)
model = get_peft_model(robert_model, config).to(device)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 399
})

In [34]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
        "eval_loss": eval_pred.loss if hasattr(eval_pred, "loss") else None  # Optional: include loss
    }

In [36]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="../modern-bert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
)

In [37]:
trainer = Trainer(
    model=roberta_model_1,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [40]:
# Ensure label column is int for all datasets
trainer.train()

W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233] WON'T CONVERT compiled_embeddings f:\python_file_install\lib\site-packages\transformers\models\modernbert\modeling_modernbert.py line 206 
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233] due to: 
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233] Traceback (most recent call last):
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233]   File "f:\python_file_install\lib\site-packages\torch\_dynamo\convert_frame.py", line 1164, in __call__
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233]     result = self._inner_convert(
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233]   File "f:\python_file_install\lib\site-packages\torch\_dynamo\convert_frame.py", line 547, in __call__
W0809 09:39:23.535000 11768 site-packages\torch\_dynamo\convert_frame.py:1233]     return _compile(


OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.16 GiB is allocated by PyTorch, and 275.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.evaluate()

f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.9955605864524841,
 'eval_accuracy': 0.635,
 'eval_f1': 0.5444059102524209,
 'eval_runtime': 26.3795,
 'eval_samples_per_second': 7.582,
 'eval_steps_per_second': 0.948,
 'epoch': 10.0}

In [ ]:
results = trainer.predict(test_dataset)

f:\python_file_install\lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
logits = results.predictions
labels = results.label_ids
metrics = compute_metrics((logits, labels))
print(metrics)

{'accuracy': 0.65, 'f1': 0.5664597115348405, 'eval_loss': None}


In [ ]:
SAVE_PATH = os.path.join(".." , "text_save_model", "modernbert_model_4")
os.makedirs(SAVE_PATH, exist_ok=True)
# robert_model.save_model(SAVE_PATH)
trainer.save_model(SAVE_PATH)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(SAVE_PATH).to(device)
model.eval()

Some weights of the model checkpoint at ..\text_save_model\xlm_robert_model_3 were not used when initializing XLMRobertaForSequenceClassification: ['classifier.out_proj.base_layer.bias', 'classifier.out_proj.base_layer.weight', 'classifier.out_proj.lora_A.default.weight', 'classifier.out_proj.lora_B.default.weight', 'roberta.encoder.layer.0.attention.self.key.base_layer.bias', 'roberta.encoder.layer.0.attention.self.key.base_layer.weight', 'roberta.encoder.layer.0.attention.self.key.lora_A.default.weight', 'roberta.encoder.layer.0.attention.self.key.lora_B.default.weight', 'roberta.encoder.layer.0.attention.self.query.base_layer.bias', 'roberta.encoder.layer.0.attention.self.query.base_layer.weight', 'roberta.encoder.layer.0.attention.self.query.lora_A.default.weight', 'roberta.encoder.layer.0.attention.self.query.lora_B.default.weight', 'roberta.encoder.layer.0.attention.self.value.base_layer.bias', 'roberta.encoder.layer.0.attention.self.value.base_layer.weight', 'roberta.encoder.lay

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
train_data["text"][0]

"I didn't feel humiliated, which was a surprise to me, considering the circumstances that had just unfolded, and as I reflected on it, I realized that it was a testament to my growth and resilience, and it made me think about all the times I had felt humiliated in the past, and how it had affected me, and I had to admit that it was a pretty terrible feeling, and one that I didn't wish on anyone, but at the same time, I was grateful that I had been able to navigate this situation without feeling that way, and it made me wonder if I had finally reached a point where I was comfortable with myself, and wasn't as concerned with what others thought of me, and that was a pretty liberating feeling, and it made me think about the concept of humiliation, and how it can be used as a tool to control and manipulate people, and how it can be used to make someone feel small and powerless, and it was interesting to me that I had been able to avoid feeling that way, even in a situation where I could ha

In [ ]:
train_data["label"][0]

np.int64(0)

In [ ]:
text = train_data["text"][0]
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs = {k: v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits

print("Predicted label id:", predictions)

Predicted label id: tensor([[ 0.1209,  0.0660, -0.1847, -0.0564,  0.3370,  0.0738]],
       device='cuda:0')
